In [121]:
import pandas as pd
import numpy as np
import os

In [122]:
# creating the dataframes
schoolsdf = pd.read_csv(os.path.join("raw_data","schools_complete.csv"))
studentsdf =  pd.read_csv(os.path.join("raw_data","students_complete.csv"))
schoolsdf.describe()


,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [123]:
# This cell is for learning about the Schools Data
#schoolsdf.describe()
schoolsdf.columns

Index(['School ID', 'name', 'type', 'size', 'budget'], dtype='object')

In [124]:
#This cell is for learningabout the Students Data
#studentsdf.describe()
studentsdf.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [125]:
studentsdf.describe()


,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [126]:
# District Summary
TotalSchools = schoolsdf['School ID'].count()
TotalStudents = studentsdf['Student ID'].count()
Totalbudget = schoolsdf['budget'].sum()
AvgMathScore = studentsdf['math_score'].mean()
AvgReadingScore = studentsdf['reading_score'].mean()
PerMathPass= (len(studentsdf.loc[studentsdf['math_score'] >65])/TotalStudents)*100
PerReadingPass = (len(studentsdf.loc[studentsdf['reading_score'] >65])/TotalStudents)*100
OverallPassingRate =(PerMathPass+PerReadingPass)/2


DistrictSummary= {'Total Schools': TotalSchools,'Total Students' : TotalStudents,'Total budget': Totalbudget,
                  'Average Math Score': AvgMathScore, 'Average Reading Score':AvgReadingScore,
                  '% Passing Math': PerMathPass, '% Passing Reading' :PerReadingPass, 'Overall Passing Rate' : OverallPassingRate}

DistrictSummaryDf = pd.DataFrame(DistrictSummary,columns = DistrictSummary.keys(),index=[0])
DistrictSummaryDf

,Total Schools,Total Students,Total budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,83.112076,94.263467,88.687771


In [133]:
#School Summary
#Final columns in the report
SummaryCol = ['School Name','School Type','Total Students','Total School Budget',
              'Per Student Budget','Average Math Score','Average Reading Score',
              '% Passing Math','% Passing Reading', 'Overall Passing Rate']


# We are creating this report in 3 steps.
#Step 1 : Group by the students dataframe by school and aggregate as many fields as you ca
schoolSumDf = studentsdf.groupby("school").agg({'reading_score':['mean'],
                                                'math_score':['mean'],
                                                'Student ID':['count']})
schoolSumDf.rename(columns = {'count': 'Total Students'})

#Step 2 : Create 2 dataframes with number of students passing math and reading
StudentMathPassDf = studentsdf.loc[studentsdf['math_score'] >65]
StudentMathPassDf=StudentMathPassDf.groupby("school").count()
StudentMathPassDf.rename(columns = {'math_score': "NumMathPassed"})

StudentReadingPassDf = studentsdf.loc[studentsdf['reading_score'] >65]
StudentReadingPassDf=StudentReadingPassDf.groupby("school").count()
StudentReadingPassDf.rename(columns = {'reading_score': "NumReadingPassed"})
                                        
StudentMathPassDf.columns



Index(['Student ID', 'name', 'gender', 'grade', 'reading_score', 'math_score'], dtype='object')